In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#setting os path
import os
os.chdir('/content/drive/MyDrive/ChatBot/Data')

In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
#loading question and answer embeddings of train and validation data from disk
train_embeds=pd.read_csv('re_train_embeds.csv')
validation_embeds=pd.read_csv('re_validation_embeds.csv')

In [ ]:
#function to preprocess question and answer embeddings post loading from disk
def preprocess_embeds(embed):
  output=[]
  embed=embed[2:-2]
  embed=embed.replace('\n','')
  for i in embed.split(' '):
    if i!=' ' and i!='':
      output.append(i)
  return (output)

In [ ]:
#preprocessing embeddings post loading from disk for train data
from tqdm.notebook import tqdm
tqdm.pandas()
train_embeds['question_embeds']=train_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
train_embeds['answer_embeds']=train_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
#preprocessing embeddings post loading from validation data
validation_embeds['question_embeds']=validation_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
validation_embeds['answer_embeds']=validation_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

In [ ]:
#function to l2 normalize each embedding
def normalize(x):
  x=np.array(x,dtype=np.float64)
  return (x)/(np.linalg.norm(x))

In [ ]:
#normalizing train question and answer embeddings
train_embeds['question_embeds_norm']=train_embeds['question_embeds'].apply(lambda x: normalize(x))
train_embeds['answer_embeds_norm']=train_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
#normalizing validation question and answer embeddings
validation_embeds['question_embeds_norm']=validation_embeds['question_embeds'].apply(lambda x: normalize(x))
validation_embeds['answer_embeds_norm']=validation_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [2]:
#Cleaning the gpt train data
train_gpt_data=train_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
train_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
train_gpt_data.head(3)

In [ ]:
os.chdir('/content/drive/MyDrive/ChatBot/Data/gpt')

In [ ]:
#saving gpt training data to disk
train_gpt_data.to_pickle("./train_gpt_data.pkl")

In [3]:
#cleaning gpt validation data
validation_gpt_data=validation_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
validation_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
validation_gpt_data.head(3)

In [ ]:
#saving the gpt validation data to disk
validation_gpt_data.to_pickle("./validation_gpt_data.pkl")

In [ ]:
#loading train and validation gpt data from disk
train_gpt_data=pd.read_pickle('./train_gpt_data.pkl')
validation_gpt_data=pd.read_pickle('./validation_gpt_data.pkl')

In [4]:
#Getting similar question and preparing actual gpt input
#installing faiss and other required libraries
!wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [5]:
!pip install faiss-gpu

In [6]:
!pip install transformers

In [ ]:
import os
import csv
from tqdm import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict

#creating faiss index for semantic search 
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
qa = pd.read_pickle('./train_gpt_data.pkl')
question_bert = qa["Q_FFNN_embeds"].tolist()
answer_bert = qa["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')
answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])
answer_index.add(answer_bert)

In [ ]:
topk=20
scores,indices=answer_index.search(
              np.array([qa['Q_FFNN_embeds'][1]]).astype('float32'), topk)
q_sub=qa.iloc[indices.reshape(20)]

line = '`QUESTION: %s `ANSWER: %s' % (
                    qa.question[1], qa.answer[1])

In [ ]:
for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')

In [ ]:
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
#defining function to prepare the actual gpt training data by retrieving similar question and answer pairs
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py

# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")

from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
def preparing_gpt_training_data(question,answer,question_embedding):
  topk=20
  scores,indices=answer_index.search(
                  np.array([question_embedding]).astype('float32'), topk)
  q_sub=qa.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: %s' % (
                        question, answer)
  encoded_len=len(tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return tokenizer.encode(line)[-1024:]

In [7]:
#preparing the gpt data for the training data
qa['gpt_data']=qa.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

In [8]:
#preparing the gpt data for the validation data
validation_gpt_data['gpt_data']=validation_gpt_data.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

In [ ]:
#function to get the start of positive mask to be used while creating the masks
def mask_start(gpt_data):
    return 1024-gpt_data[::-1].index(4600)+1

In [9]:
#finding the mask start for the train dataset
qa['mask_start']=qa.gpt_data.progress_apply(lambda x: mask_start(x))

In [10]:
#finding the mask start for the validation dataset
validation_gpt_data['mask_start']=validation_gpt_data.gpt_data.progress_apply(lambda x: mask_start(x))

In [ ]:
#saving train data to disk
qa.to_pickle('./qa_gpt_inbuild.pkl')

In [ ]:
#finding length of the gpt data for train dataset
qa['gpt_lens']=qa.gpt_data.apply(lambda x: len(x))

In [ ]:
#finding length of the gpt data for validation dataset
validation_gpt_data['gpt_lens']=validation_gpt_data.gpt_data.apply(lambda x: len(x))

In [11]:
#finding train datapoints that has sequence length of 1024
gpt_data_cleaned=qa[qa.gpt_lens==1024]
gpt_data_cleaned.shape

In [12]:
#finding validation datapoints that has sequence length of 1024
validation_gpt_data_cleaned=validation_gpt_data[validation_gpt_data.gpt_lens==1024]
validation_gpt_data_cleaned.shape

In [ ]:
#function to create the loss mask given mask start
def return_loss_mask(mask_start):
  return [0]*mask_start+[1]*(1024-mask_start)

In [ ]:
#creating loss mask for train data
gpt_data_cleaned['mask']=gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#creating loss mask for validation data
validation_gpt_data_cleaned['mask']=validation_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

In [ ]:
#saving train gpt data with loss mask to disk
gpt_data_cleaned.to_pickle('./qa_gpt_inbuild_cleaned.pkl')

In [ ]:
#saving validation gpt data with loss mask to disk
validation_gpt_data_cleaned.to_pickle('./validation_gpt_inbuild_cleaned.pkl')

In [ ]:
#loading gpt train and validation data from disk
gpt_data_cleaned=pd.read_pickle('./qa_gpt_inbuild_cleaned.pkl')
validation_gpt_data_cleaned=pd.read_pickle('./validation_gpt_inbuild_cleaned.pkl')
gpt_data_cleaned.shape,validation_gpt_data_cleaned.shape

((23797, 8), (5950, 8))

In [ ]:
#extracting gpt data for train data
gpt_data=[]
for i in gpt_data_cleaned.gpt_data.values:
  gpt_data.append(i)
gpt_data=np.array(gpt_data)
gpt_data.shape

In [ ]:
#extracting gpt data for validation data
validation_gpt_data=[]
for i in validation_gpt_data_cleaned.gpt_data.values:
  validation_gpt_data.append(i)
validation_gpt_data=np.array(validation_gpt_data)
validation_gpt_data.shape

In [ ]:
len(gpt_data_cleaned['mask'][0])

In [ ]:
#extracting loss mask for train data
loss_mask=[]
for i in gpt_data_cleaned['mask'].values:
  loss_mask.append(i)
loss_mask=np.array(loss_mask)
loss_mask.shape

(23797, 1024)

In [ ]:
#extracting loss mask for validation data
validation_loss_mask=[]
for i in validation_gpt_data_cleaned['mask'].values:
  validation_loss_mask.append(i)
validation_loss_mask=np.array(validation_loss_mask)
validation_loss_mask.shape

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt train dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': gpt_data,

        'label': gpt_data[:,1:],
        'loss_mask':loss_mask[:,:-1]
    },
))


gpt_train_dataset = gpt_train_dataset.shuffle(BUFFER_SIZE)
gpt_train_dataset = gpt_train_dataset.batch(BATCH_SIZE)
gpt_train_dataset = gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt validation dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
validation_gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': validation_gpt_data,

        'label': validation_gpt_data[:,1:],
        'loss_mask':validation_loss_mask[:,:-1]
    },
))

validation_gpt_train_dataset = validation_gpt_train_dataset.shuffle(BUFFER_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.batch(BATCH_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#loading pre-trained gpt2 tokenizer and gpt2 model
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#displaying sample gpt training data
next(iter(gpt_train_dataset))

In [1]:
#training the gpt2 model
learning_rate=0.0001
optim=tf.optimizers.Adam(learning_rate)
j=0
loss_cum=0
epoch_loss=0
stop_training=False
epochs=10
for epoch in range(epochs):
  print('epoch',epoch)
  for i in tqdm(gpt_train_dataset):
    with tf.GradientTape() as tape:
      context=i[0]['context']
      loss_mask=tf.cast(i[0]['loss_mask'],tf.float32)
      output=tf_gpt2_model(context,training=True)['logits']
      loss_value = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=np.array(i[0]['label']), logits=output[ :,:-1])*loss_mask)
      j=j+1
      loss_cum+=loss_value
      epoch_loss+=loss_value/100
      if j%50==0:
        print(loss_cum/50)
        grads=tape.gradient(loss_cum/50,tf_gpt2_model.trainable_variables)
        optim.apply_gradients(zip(grads,tf_gpt2_model.trainable_variables))
        loss_cum=0
      if tf.math.is_nan(loss_value):
        print('loss became nan')
        stop_training=True
        break
      if j%10000==0:
        tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch)+'_'+str(j))
  print('epoch_loss by 100',epoch_loss/237)
  epoch_loss=0
  if stop_training:
    break
  tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch))

In [ ]:
#loading the gpt2 model from the above saved checkpoint
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("./tf_gpt2_model_2_106_160000")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./tf_gpt2_model_2_106_160000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/ChatBot/Data'

In [ ]:
#function to return sample origingal and generated answers for the train dataset
def return_sample_output_predicted():
  sample_data=next(iter(gpt_train_dataset))
  v=sample_data[0]['context'].numpy()[0]
  mask_start = len(v) - list(v[::-1]).index(4600) + 1
  input=v[:mask_start+1]
  next_step_input=[]
  input1=list(input)+next_step_input
  final_output=[]
  for i in range(0,1024-len(input)):
    input1=list(input1)+next_step_input
    output=tf_gpt2_model([np.array(input1)])['logits']
    output1=np.argmax(output,axis=1)
    next_step_input=[output1[-1]]#appending last output to input
    next_step_char=tokenizer.decode(next_step_input)
    final_output.append(next_step_char)
  expected_output=v[mask_start+1:]
  return v,tokenizer.decode(expected_output),' '.join(final_output)

In [ ]:
#displaying sample original context, expected answer and generated answer for train datset
for i in range(5):
  v,exp,fin=return_sample_output_predicted()
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 scheduled for awhile should i call for an immediate visit now or wait until tomorrow i am diabetic with high blood pressure and cholesterol all medicated what did i experience `ANSWER: this happened to me a few years ago i was so scared that i was at the eye doctor is office when it opened they worked me in and dr gave some fancy name for it and said nothing to worry about it never happened again it was worth the trip just to be re assured `QUESTION: diagnosis  hidradenitis suppurativa  large and painful boils in the groin area considering injections success rate  this condition started 2 years ago and exacerbated with shaving it is extremely painful cannot be improved with excessive hygiene is embarrassing has resulted in multiple scars and had a psychological effect on me also i am considering the injections my dr said it is botox but i read that it is antibiotic i just want it to go away sometimes the boils last 2 3 weeks can it be permanently cured or go into a remission period it

In [ ]:

#function to return sample origingal and generated answers for the validation dataset
def return_validation_sample_output_predicted():
  sample_data=next(iter(validation_gpt_train_dataset))
  v=sample_data[0]['context'].numpy()[0]
  mask_start = len(v) - list(v[::-1]).index(4600) + 1
  input=v[:mask_start+1]
  next_step_input=[]
  input1=list(input)+next_step_input
  final_output=[]
  token_output=[]
  for i in range(0,1024-len(input)):
    input1=list(input1)+next_step_input
    output=tf_gpt2_model([np.array(input1)])['logits']
    output1=np.argmax(output,axis=1)
    next_step_input=[output1[-1]]#appending last output to input
    next_step_char=tokenizer.decode(next_step_input)
    token_output.append(next_step_input[0])
    final_output.append(next_step_char)
  expected_output=v[mask_start+1:]
  return v,tokenizer.decode(expected_output),tokenizer.decode(token_output)

In [ ]:
#displaying sample original context, expected answer and generated answer for validation datset
for i in range(5):
  v,exp,fin=return_validation_sample_output_predicted()
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 trauma the doctors now are saying not to take too much calcium suplements `QUESTION: what causes a sudden onset of vertigo while lying down or turning while lying down  within the last two weeks i have begun to experience sudden vertigo while lying down or just turning while lying down if i try to stand up i am completely disoriented i do not understand what is causing this i am not on any medication do not drink or smoke what could be causing this `ANSWER: in the last week i have begun to experience sudden vertigo while lying down if i try to to stand up i become completely disoriented what could be causing this i do take lisinopril 20mg and terazosin 5mg for prostrate do not smoke or drink `QUESTION: can a methylprednisolone injection in the spine cause muscle spasms  i had an steroid methylprednisolone injection in the sacroiliac joint 3 days ago and am now having severe muscle spasms at night mostly in the legs but in my fingers as well does this steroid have anything to do with t